In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
import seaborn as sns
import sklearn
import matplotlib.pyplot as plt
import keras
from sklearn.model_selection import train_test_split
from kaggle_datasets import KaggleDatasets
import cv2
import os

In [ ]:
PATH_DIR ="../input/plant-pathology-2021-fgvc8"

TRAIN_DIR = "../input/plant-pathology-2021-fgvc8/train_images/"

TEST_DIR = "../input/plant-pathology-2021-fgvc8/test_images/"


train_label =pd.read_csv('../input/plant-pathology-2021-fgvc8/train.csv')


In [ ]:
train_label

In [ ]:
train_label['labels'].unique()

In [ ]:
train_label['labels'].value_counts()

In [ ]:
def plot_examples(label):
    fig, ax = plt.subplots(1, 6, figsize=(25, 12))
    ax = ax.ravel()
    for i in range(6):
        idx = train_label[train_label['labels']==label].index[i]
        image = cv2.imread(TRAIN_DIR+train_label.loc[idx, 'image'])
        
        image =cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        ax[i].imshow(image)
        ax[i].set_title(label)
        ax[i].set_xticklabels([])
        ax[i].set_yticklabels([])

In [ ]:
for labels in list(train_label['labels'].unique()):
    plot_examples(labels)

# **Play With OpenCV**

### **Sharpen image**

In [ ]:
def plot_canny(label):
    fig, ax = plt.subplots(3,2, figsize=(20, 15))
    ax = ax.ravel()
    for i in range(6):
        amount=1.0; threshold=0.4
        alpha =1.0;
        beta= 0.6
        idx = train_label[train_label['labels']==label].index[i]
        image = cv2.imread(TRAIN_DIR+train_label.loc[idx, 'image'])
        
        image =cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        
        
        blurred = cv2.GaussianBlur(image, (5,5), sigmaX=1.0)
        sharpened = float(amount + 1) * image - float(amount) * blurred
        sharpened = np.maximum(sharpened, np.zeros(sharpened.shape))
        sharpened = np.minimum(sharpened, 255 * np.ones(sharpened.shape))
        sharpened = sharpened.round().astype(np.uint8)
        if threshold > 0:
            low_contrast_mask = np.absolute(image - blurred) < threshold
            np.copyto(sharpened, image, where=low_contrast_mask)
        ax[i].imshow(sharpened)
        ax[i].set_title(label)
        ax[i].set_xticklabels([])
        ax[i].set_yticklabels([])
        
plot_canny(label="complex")

### **Canny Edge**

In [ ]:
def plot_contour(label):
    fig, ax = plt.subplots(3,2, figsize=(20, 15))
    ax = ax.ravel()
    for i in range(6):
        amount=1.0; threshold=0.4
        alpha =1.0;
        beta= 0.6
        idx = train_label[train_label['labels']==label].index[i]
        image = cv2.imread(TRAIN_DIR+train_label.loc[idx, 'image'])
        #increase brighness
        
        hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
        h, s, v = cv2.split(hsv)

        lim = 255 - 20
        v[v > lim] = 255
        v[v <= lim] += 20

        final_hsv = cv2.merge((h, s, v))
        image = cv2.cvtColor(final_hsv, cv2.COLOR_HSV2BGR)
        
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        img_gray = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)
        ret, thresh = cv2.threshold(img_gray, 200, 255, 1)
        contours, hierarchy = cv2.findContours(thresh, cv2.RETR_TREE, cv2.CHAIN_APPROX_NONE)
        
        
        image= cv2.drawContours(image, contours, -1, (0,0,0), 2)
        image = cv2.Canny(image, 100, 200)
        
        ax[i].imshow(image)
        ax[i].set_title(label)
        ax[i].set_xticklabels([])
        ax[i].set_yticklabels([])
        
plot_contour(label="scab frog_eye_leaf_spot complex")

### **Contour**

In [ ]:
def plot_contour(label):
    fig, ax = plt.subplots(1,4, figsize=(20, 15))
    ax = ax.ravel()
    for i in range(6):
        amount=1.0; threshold=0.4
        alpha =1.0;
        beta= 0.6
        idx = train_label[train_label['labels']==label].index[i]
        image = cv2.imread(TRAIN_DIR+train_label.loc[idx, 'image'])
        
        image =cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        
        
        blurred = cv2.GaussianBlur(image, (5,5), sigmaX=1.0)
        sharpened = float(amount + 1) * image - float(amount) * blurred
        sharpened = np.maximum(sharpened, np.zeros(sharpened.shape))
        sharpened = np.minimum(sharpened, 255 * np.ones(sharpened.shape))
        sharpened = sharpened.round().astype(np.uint8)
        if threshold > 0:
            low_contrast_mask = np.absolute(image - blurred) < threshold
            np.copyto(sharpened, image, where=low_contrast_mask)
            
            
        img_gray = cv2.cvtColor(sharpened, cv2.COLOR_RGB2GRAY)
        ret, thresh = cv2.threshold(img_gray, 200, 255, 1)
        contours, hierarchy = cv2.findContours(thresh, cv2.RETR_TREE, cv2.CHAIN_APPROX_NONE)
        
        
        image= cv2.drawContours(sharpened, contours, -1, (255,0,0), 2)
        
        ax[i].imshow(image)
        ax[i].set_title(label)
        ax[i].set_xticklabels([])
        ax[i].set_yticklabels([])
        
for label in list(train_label['labels'].unique()):        
    plot_contour(label )